# Background subtraction

This notebook implements the method developed in script Background.ipynb in a loop to handle a batch of images.

It expects .ARW images as input, and generates corresponding .bkg_subtracted.fits files.

In [1]:
import os, glob
import time, datetime
import math

import multiprocessing as mp
from multiprocessing import Pool

import numpy as np

from scipy import stats

from astropy.io import fits
from astropy.table import Table
from astropy.stats import SigmaClip
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans

from photutils.background import Background2D, MedianBackground, ModeEstimatorBackground

import rawpy
import exifread

from worker_background import Worker
from datapath import DATA

## Initialization

In [2]:
# these were determined in script White_light_images as the normalization
# factors that render the smoothest background.
red_norm =  1.34
blue_norm = 1.25

# parameters to control background subtraction
bkg_cell_footprint = (100, 100)
bkg_filter = (11, 11)

bkg_sigma_clip = SigmaClip(sigma=5.)
bkg_kernel = Gaussian2DKernel(x_stddev=1)
bkg_estimator = ModeEstimatorBackground()

In [3]:
# paths
data_dirpath = os.path.join(DATA,'astrophotography_data/Andromeda_2022/135mm1s3200ISO')

input_suffix = '.ARW'
output_suffix = '.bkg_subtracted.fits'

In [4]:
# images to subtract background from
image_list = list(glob.glob(data_dirpath + '/*' + input_suffix))
image_list.sort()

image_list

['/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00362.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00363.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00364.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00365.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00366.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00367.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00368.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00369.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00370.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00371.ARW',
 '/Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00372.ARW',

In [5]:
# read base image - we need this in order to access the camera color matrix
raw = rawpy.imread(image_list[int(len(image_list)/2)]) # mid-point
imarray_base = raw.raw_image_visible.astype(float)

In [6]:
# masks that isolate the RGB pixels
colors_array = raw.raw_colors_visible

red_mask = np.where(colors_array == 0, 1, 0)

green_mask_1 = np.where(colors_array == 1, 1, 0)
green_mask_2 = np.where(colors_array == 3, 1, 0)
green_mask = green_mask_1 | green_mask_2

blue_mask = np.where(colors_array == 2, 1, 0)

## Functions

All functions are defined in module 'worker_background.py'. The multiprocessing framework requires that everything that runs on a separate process has its own address space. 

## Main loop

In [7]:
# for image_name in image_list:
#     raw = rawpy.imread(image_name)
#     imarray = raw.raw_image_visible.astype(float)
#     subtracted = subtract_background(imarray, red_norm=red_norm, blue_norm=blue_norm)
#     write_bkg_subtracted(image_name, subtracted)

# number of processors
nproc = 8    # (Mac M1)

# split file list in nproc chuncks; one for each processor
image_list_chuncks = np.array_split(image_list, nproc)

results = []
pool = Pool(nproc)

for p in range(nproc):
    worker = Worker(image_list_chuncks[p], input_suffix, output_suffix, 
                    red_mask, green_mask, blue_mask, 
                    red_norm, blue_norm,
                    bkg_cell_footprint, bkg_filter, bkg_sigma_clip, bkg_kernel, bkg_estimator)

    r = pool.apply_async(worker)
    results.append(r)

for r in results:
    r.wait()

pool.close()    

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00588.bkg_subtracted.fits Sat Sep  9 10:58:24 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00701.bkg_subtracted.fits Sat Sep  9 10:58:24 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00362.bkg_subtracted.fits Sat Sep  9 10:58:25 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00926.bkg_subtracted.fits Sat Sep  9 10:58:25 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00814.bkg_subtracted.fits Sat Sep  9 10:58:26 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00475.bkg_subtracted.fits Sat Sep  9 10:58:26 2023
Writing bkg-subtracted image:  /Users/busko/Pr

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00932.bkg_subtracted.fits Sat Sep  9 11:00:32 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00820.bkg_subtracted.fits Sat Sep  9 11:00:40 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00368.bkg_subtracted.fits Sat Sep  9 11:00:44 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00595.bkg_subtracted.fits Sat Sep  9 11:00:47 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01157.bkg_subtracted.fits Sat Sep  9 11:00:53 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00933.bkg_subtracted.fits Sat Sep  9 11:00:53 2023
Writing bkg-subtracted image:  /Users/busko/Pr

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00939.bkg_subtracted.fits Sat Sep  9 11:02:59 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01051.bkg_subtracted.fits Sat Sep  9 11:02:59 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00826.bkg_subtracted.fits Sat Sep  9 11:03:00 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00374.bkg_subtracted.fits Sat Sep  9 11:03:01 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00602.bkg_subtracted.fits Sat Sep  9 11:03:06 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01164.bkg_subtracted.fits Sat Sep  9 11:03:11 2023
Writing bkg-subtracted image:  /Users/busko/Pr

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00946.bkg_subtracted.fits Sat Sep  9 11:04:58 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00832.bkg_subtracted.fits Sat Sep  9 11:04:59 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01058.bkg_subtracted.fits Sat Sep  9 11:05:00 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00721.bkg_subtracted.fits Sat Sep  9 11:05:01 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00492.bkg_subtracted.fits Sat Sep  9 11:05:03 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00609.bkg_subtracted.fits Sat Sep  9 11:05:06 2023
Writing bkg-subtracted image:  /Users/busko/Pr

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00838.bkg_subtracted.fits Sat Sep  9 11:06:56 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00953.bkg_subtracted.fits Sat Sep  9 11:06:57 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00616.bkg_subtracted.fits Sat Sep  9 11:07:05 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00498.bkg_subtracted.fits Sat Sep  9 11:07:06 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01065.bkg_subtracted.fits Sat Sep  9 11:07:07 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00728.bkg_subtracted.fits Sat Sep  9 11:07:07 2023
Writing bkg-subtracted image:  /Users/busko/Pr

...

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00893.bkg_subtracted.fits Sat Sep  9 11:25:04 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00549.bkg_subtracted.fits Sat Sep  9 11:25:06 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00678.bkg_subtracted.fits Sat Sep  9 11:25:09 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01239.bkg_subtracted.fits Sat Sep  9 11:25:13 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00789.bkg_subtracted.fits Sat Sep  9 11:25:15 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01015.bkg_subtracted.fits Sat Sep  9 11:25:15 2023
Writing bkg-subtracted image:  /Users/busko/Pr

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00447.bkg_subtracted.fits Sat Sep  9 11:27:00 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00899.bkg_subtracted.fits Sat Sep  9 11:27:00 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00555.bkg_subtracted.fits Sat Sep  9 11:27:08 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00685.bkg_subtracted.fits Sat Sep  9 11:27:10 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01022.bkg_subtracted.fits Sat Sep  9 11:27:14 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00796.bkg_subtracted.fits Sat Sep  9 11:27:14 2023
Writing bkg-subtracted image:  /Users/busko/Pr

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01139.bkg_subtracted.fits Sat Sep  9 11:29:03 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00802.bkg_subtracted.fits Sat Sep  9 11:29:03 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01252.bkg_subtracted.fits Sat Sep  9 11:29:03 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00692.bkg_subtracted.fits Sat Sep  9 11:29:14 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00561.bkg_subtracted.fits Sat Sep  9 11:29:17 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01140.bkg_subtracted.fits Sat Sep  9 11:29:19 2023
Writing bkg-subtracted image:  /Users/busko/Pr

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01259.bkg_subtracted.fits Sat Sep  9 11:31:09 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01146.bkg_subtracted.fits Sat Sep  9 11:31:09 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00809.bkg_subtracted.fits Sat Sep  9 11:31:09 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC01035.bkg_subtracted.fits Sat Sep  9 11:31:09 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00699.bkg_subtracted.fits Sat Sep  9 11:31:16 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00912.bkg_subtracted.fits Sat Sep  9 11:31:19 2023
Writing bkg-subtracted image:  /Users/busko/Pr

Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00578.bkg_subtracted.fits Sat Sep  9 11:33:47 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00924.bkg_subtracted.fits Sat Sep  9 11:33:49 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00472.bkg_subtracted.fits Sat Sep  9 11:33:50 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00579.bkg_subtracted.fits Sat Sep  9 11:33:59 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00925.bkg_subtracted.fits Sat Sep  9 11:34:01 2023
Writing bkg-subtracted image:  /Users/busko/Projects/astrophotography_data/Andromeda_2022/135mm1s3200ISO/DSC00473.bkg_subtracted.fits Sat Sep  9 11:34:01 2023
Writing bkg-subtracted image:  /Users/busko/Pr